# Fine-tuning a Pre-trained Transformer for Document Classification

In this lab you will fine-tune a pre-trained transformer model using the huggingface `transformers` library. This library provides a number of transformer models such as BERT, XLNet, and GPT, that can be used with PyTorch or Tensorflow. The tokenisers for these models are also included, which makes using transformers with this library much easier than developing them from scratch.

The [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) which is a smaller version of BERT will be used in this lab. It is much faster to fine-tune, but will give slightly worse performance than the original BERT model.

For the dataset we will use the [IMDB movie review data](https://huggingface.co/datasets/imdb) where the task is to classify a review as either positive if the reviewer liked the movie or negative if the reviewer did not like the movie. The input is the text of the review and the output is a binary label either 0 (negative) or 1 (positive).
In previous labs and the assignments, you have explored this dataset multiple times but with a differnt train/validation/test splits. 

We will use both `pytorch` and the `transformers` library, as well as a few other useful libraries such as `tqdm` to make progress bars, and `sklearn` for its evaluation metric.

We will also use the `datasets` library which is the huggingface datasets library and can be installed using `pip`.

In [1]:
!pip install transformers datasets ipywidgets

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------- ----------- 41.0/57.6 kB 1.9 MB/s eta 0:00:01
     ---------------------------- ----------- 41.0/57.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 430.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/52.5 kB ? eta -:--:--
     ---------------------------------------- 52.5/52.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/471.6 kB ? eta -:--:--
   --------- ------------------------------ 112.6/471.6 kB ? eta -:--:--
   ------------------------------ --------- 358.4/471.6 kB 3.7 MB/s eta 0:00:01
   ------------------------------ --------- 358.4/471.6 kB 3.7 MB/s eta 0:00:01
   ------------------------------ --------- 358.4/471.6 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 471.6/471.6 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/380.7 kB ? eta -:--:--
   -----

In [3]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from datasets import load_dataset
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
import numpy as np
from pprint import pprint

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

First, load the `imdb` data using the huggingface `datasets` library, which will automatically download the data if it is not already cached on your system.

In [5]:
data = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

C:\Users\apex\anaconda3\envs\mldl\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\apex\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

If you print the `data` variable you will see that you have a dictionary like object with keys `test`, `train`, `unsupervised`, and values of `TorchIterableDataset`. We will only use the `test` and `train` data for this lab. You can inspect the first element of the data as follows:

In [6]:
pprint(data['train'][0])

{'label': 0,
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the '
         'controversy that surrounded it when it was first released in 1967. I '
         'also heard that at first it was seized by U.S. customs if it ever '
         'tried to enter this country, therefore being a fan of films '
         'considered "controversial" I really had to see this for myself.<br '
         '/><br />The plot is centered around a young Swedish drama student '
         'named Lena who wants to learn everything she can about life. In '
         'particular she wants to focus her attentions to making some sort of '
         'documentary on what the average Swede thought about certain '
         'political issues such as the Vietnam War and race issues in the '
         'United States. In between asking politicians and ordinary denizens '
         'of Stockholm about their opinions on politics, she has sex with her '
         'drama teacher, classmates, and married men.<br

If you do this you will get a dictionary with both the `label` and the `text` of the first element. The label indicates if this text is a positive or negative review.

### Tokeniser

Since the `text` is currently a string of characters we need to:
1. Split the sequence into tokens (i.e. word-pieces for BERT models but will be different depending on the pre-trained model)
2. Represent the sequence as token/word-piece ids
3. Add a `[CLS]` token to the start of the sequence, and a `[SEP]` token to the end
4. Pad the sequences with `0`'s so they are all the same length
5. Construct an attention mask for the input (to identify which parts of the input are padding and so should be ignored)

Fortunately, the `transformers` library provides an easy way to do all of this.

First, get the tokeniser specific to DistilBert ([documentation here](https://huggingface.co/docs/transformers/model_doc/distilbert)). \
This will download the tokeniser for the lowercase only version of DistilBert. The download is only a few kilobytes. 

In [7]:
model_name = "distilbert-base-uncased"
tok = DistilBertTokenizerFast.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\apex\anaconda3\envs\mldl\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\apex\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

C:\Users\apex\anaconda3\envs\mldl\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Next, write a function to apply the tokeniser to the `text` field in the dataset.

This will do all the steps 1-5 listed above and return the attention mask and the sequence of ids. The arguments indicate we are truncating sequences longer than the maximum length, and padding all sequences that are less than the maximum length so that they are exactly the maximum length. The maximum length for the DistilBert model is 512 word-pieces.

In [8]:
def tokenize_fn(X):
    return tok(X["text"], truncation = True, padding="max_length")

### Training data

Now we can apply the tokeniser to the text as follows:

In [9]:
shuffle_train_data = data['train'].shuffle(seed=42)
tokenized_train_data = shuffle_train_data.map(tokenize_fn, batched=True)
small_train_data = tokenized_train_data.select(range(1000))
small_train_data = small_train_data.with_format("torch", columns=["label", "input_ids", "attention_mask"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

The part which actually applies the tokeniser is the `map` call. This will call the `tokenize_fn` function on each of the examples in `data['train']`.

The `shuffle` ensures the data is in a random order (if you don't do this you will run into problems because the dataset has all the 0 classes first followed by all the 1 classes).

The `select` statement just extracts the first 1000 examples (of the shuffled list), if you were training this model to get the best performance you would use all the examples but for this lab we will use only 1000 to avoid waiting around for the model to train.

The final line which calls `with_format` is responsible for converting the data columns `label`, `input_ids`, and `attention_mask` into PyTorch tensors.

It is worthwhile to inspect `small_train_data` at this point. To do this use:

In [12]:
print(small_train_data[0])

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0

You will see that the datapoint has an `attention_mask`, `input_ids`, and `label`. The `attention_mask` and `input_ids` are new and were added by `tokenize_fn`.

The `input_ids` are the padded sequences of word-piece ids, while the `attention_mask` identifies which parts of the `input_ids` are padding and so should be ignored by the attention layer. 

Note that the first id in the tensor of `input_ids` is `101`, which represents the `[CLS]` token, while the last non-zero id is `102` which is the `[SEP]` token. If you want to explore a bit further the mapping from tokens to ids can be accessed through the dictionary `tok.vocab`.

### Validation data

We can now create a validation dataset from the `test` data in the same way:

In [13]:
shuffle_val_data = data['test'].shuffle(seed=42)
tokenized_val_data = shuffle_val_data.map(tokenize_fn, batched=True)
small_val_data = tokenized_val_data.select(range(1000))
small_val_data = small_val_data.with_format("torch", columns=["label", "input_ids", "attention_mask"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

## Model
Defining and downloading the pre-trained transformer is straightforward but be aware that the download is ~270M:

In [14]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The above specifies downloads and sets-up a pre-trained `DistilBert` model and configures it for the sequence classification task. 

The download is specifically the [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) pre-trained model, which only considers lowercase characters (there is a cased version available also `distilbert-base-cased`). 

Under the hood, the huggingface sequence classifier model will feed the output of the transformer (the output value at the position of the `[CLS]` token) to a new Linear layer to get logits (un-normalised scores for each of the classes). We need to provide the number of classes so the linear layers weight matrix can be properly specified. The way to do this is with the `num_labels` argument. As there are only two classes in our task (positive and negative) we specify `2` as the number of labels.

The loaded `DistilBert` model is in the variable `model` which is also a PyTorch module with the normal set of PyTorch methods, such as `forward`, `parameters`, and `to`.

We first move the model to the desired device such as the GPU (if available) using `model.to(device)`.

In [15]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Fine-tuning

We initialize an `AdamW` optimiser, and then start the main training loop. Which consists of a model forward pass, followed by a backward pass and optimiser step. 

Note that the model takes a number of arguments:
- `labels` (optional) which are the ground truth labels (for calculating the loss function when training),
- `input_ids` which are the padded sequences of word-pieces, and
- `attention_mask` which is an binary mask indicating which parts of the inputs are actual tokens and which are padding tokens (we do not want the transformer to pay attention to any padding tokens);

and returns a [SequenceClassifierOutput](https://huggingface.co/docs/transformers/main/en/main_classes/output#transformers.modeling_outputs.SequenceClassifierOutput) object.

In [16]:
def finetune_model(model, dataset_train, dataset_val=None, eval_fn=None, batch_size=8, n_epochs=2, learning_rate=1e-5):
    model.train(True)
    
    # create a pytorch data loaders to make it easy to iterate over batches of training data
    # see https://pytorch.org/docs/stable/data.html
    dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

    n_batches = (len(dataset_train) - 1) // batch_size + 1
    
    # get the AdamW optimizer
    optimiser = torch.optim.AdamW(params=model.parameters(), lr=learning_rate)

    # run the training loop
    print(f'{"Epoch":>10} {"Batch":>20} {"Loss":>10}')
    for epoch in range(n_epochs):
        for (b, batch) in enumerate(tqdm(dataloader)):
    
            # run the transformer forwards
            outputs = model(
                labels = batch["label"].to(device),
                input_ids = batch["input_ids"].to(device),
                attention_mask = batch["attention_mask"].to(device),
            )
    
            # get the classification loss
            loss = outputs.loss
            
            # backpropagate then apply the optimiser
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
    
            # print the loss
            if (b+1) % 5 == 0:
                print(f'{epoch+1:>10} {f"{b+1} / {n_batches}":>20} {format(loss.cpu().item(), ".3f"):>10}')

        # evaluate the model on validation data
        if (dataset_val is not None) and (eval_fn is not None):
            print('Evaluating ...')
            f1 = eval_fn(model, dataset_val)
            print(f'Epoch {epoch+1}: F1 score (validation) = {format(f1, ".3f")}') 

    return model

### Exercise

Implement a `evaluate_model` function which returns the F1 score of the model on the validation data. This is going to be similar to the evaluation functions you have seen in previous labs/assignments and also reasonably similar to the training loop provided above.

In [17]:
from sklearn.metrics import precision_score, recall_score


def evaluate_model(model, dataset_val, batch_size=8):
    # TODO: Implement this function which returns the F1 score of the model on the validation data.
    dataloader = DataLoader(dataset_val, batch_size=batch_size)
    true_labels = []
    predictions = []

    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 关闭梯度计算
        for batch in dataloader:
            inputs = {
                'input_ids': batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device)
            }
            outputs = model(**inputs)
            logits = outputs.logits
            pred_labels = torch.argmax(logits, dim=-1)
            
            predictions.extend(pred_labels.cpu().numpy())
            true_labels.extend(batch['label'].numpy())

    # 计算精确率、召回率和F1分数
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')
    
    print(f'Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}')
    return f1

Fine tune the model and evaluate it on the validation set after each training epoch (this may take ~30 minutes if using CPU).

In [18]:
model = finetune_model(model, small_train_data, small_val_data, eval_fn=evaluate_model, n_epochs=3)

     Epoch                Batch       Loss


  0%|          | 0/125 [00:00<?, ?it/s]

         1              5 / 125      0.687
         1             10 / 125      0.672
         1             15 / 125      0.676
         1             20 / 125      0.693
         1             25 / 125      0.698
         1             30 / 125      0.688
         1             35 / 125      0.663
         1             40 / 125      0.662
         1             45 / 125      0.647
         1             50 / 125      0.565
         1             55 / 125      0.500
         1             60 / 125      0.439
         1             65 / 125      0.631
         1             70 / 125      0.541
         1             75 / 125      0.651
         1             80 / 125      0.508
         1             85 / 125      0.444
         1             90 / 125      0.257
         1             95 / 125      0.585
         1            100 / 125      0.206
         1            105 / 125      0.307
         1            110 / 125      0.522
         1            115 / 125      0.450
         1 

  0%|          | 0/125 [00:00<?, ?it/s]

         2              5 / 125      0.388
         2             10 / 125      0.454
         2             15 / 125      0.261
         2             20 / 125      0.083
         2             25 / 125      0.372
         2             30 / 125      0.341
         2             35 / 125      0.153
         2             40 / 125      0.137
         2             45 / 125      0.106
         2             50 / 125      0.068
         2             55 / 125      0.371
         2             60 / 125      0.304
         2             65 / 125      0.174
         2             70 / 125      0.196
         2             75 / 125      0.109
         2             80 / 125      0.259
         2             85 / 125      0.364
         2             90 / 125      0.533
         2             95 / 125      0.269
         2            100 / 125      0.081
         2            105 / 125      0.465
         2            110 / 125      0.116
         2            115 / 125      0.103
         2 

  0%|          | 0/125 [00:00<?, ?it/s]

         3              5 / 125      0.040
         3             10 / 125      0.558
         3             15 / 125      0.035
         3             20 / 125      0.036
         3             25 / 125      0.589
         3             30 / 125      0.036
         3             35 / 125      0.041
         3             40 / 125      0.455
         3             45 / 125      0.028
         3             50 / 125      0.030
         3             55 / 125      0.051
         3             60 / 125      0.035
         3             65 / 125      0.045
         3             70 / 125      0.047
         3             75 / 125      0.052
         3             80 / 125      0.186
         3             85 / 125      0.022
         3             90 / 125      0.022
         3             95 / 125      0.025
         3            100 / 125      0.032
         3            105 / 125      0.024
         3            110 / 125      0.030
         3            115 / 125      0.046
         3 